In [ ]:
# Clone the YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5
%cd yolov5

# Install requirements
!pip install -r requirements.txt

!pip install kaggle

from google.colab import drive
drive.mount('/content/drive')

Cloning into 'yolov5'...
remote: Enumerating objects: 17067, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 17067 (delta 24), reused 28 (delta 12), pack-reused 17022 (from 1)
Receiving objects: 100% (17067/17067), 15.68 MiB | 17.83 MiB/s, done.
Resolving deltas: 100% (11719/11719), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 896.9/896.9 kB 24.6 MB/s eta 0:00:00
Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/traffic


/content/drive/MyDrive/traffic


In [ ]:
!kaggle datasets download valentynsichkar/traffic-signs-dataset-in-yolo-format

Dataset URL: https://www.kaggle.com/datasets/valentynsichkar/traffic-signs-dataset-in-yolo-format
License(s): other
traffic-signs-dataset-in-yolo-format.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip traffic-signs-dataset-in-yolo-format.zip

Archive:  traffic-signs-dataset-in-yolo-format.zip
replace classes.names? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace getting-full-path.py? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace test.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace traffic-sign-to-test.mp4? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace train.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
  inflating: ts/ts/00000.jpg         
  inflating: ts/ts/00000.txt         
  inflating: ts/ts/00001.jpg         
  inflating: ts/ts/00001.txt         
  inflating: ts/ts/00002.jpg         
  inflating: ts/ts/00002.txt         
  inflating: ts/ts/00003.jpg         
  inflating: ts/ts/00003.txt         
  inflating: ts/ts/00004.jpg         
  inflating: ts/ts/00004.txt         
  inflating: ts/ts/00005.jpg         
  inflating: ts/ts/00005.txt         
  inflating: ts/ts/00006.jpg         
  inflating: ts/ts/00006.txt         
  inflating: ts/ts/00007.jpg         
  inflating: ts/ts/00007.txt         
  inflating: ts/t

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

data_dir = "/content/drive/MyDrive/traffic/ts/ts"
images_dir = os.path.join(data_dir, "images")
labels_dir = os.path.join(data_dir, "labels")

os.makedirs(images_dir, exist_ok=True)
os.makedirs(labels_dir, exist_ok=True)

# Move images and labels to respective folders
for filename in os.listdir(data_dir):
    if filename.endswith(".jpg"):
        shutil.move(os.path.join(data_dir, filename), images_dir)
    elif filename.endswith(".txt"):
        shutil.move(os.path.join(data_dir, filename), labels_dir)

# Split data into train, validation, and test sets
image_files = os.listdir(images_dir)
train_images, test_images = train_test_split(image_files, test_size=0.2, random_state=42)
train_images, val_images = train_test_split(train_images, test_size=0.2, random_state=42)

# Create train, validation, and test directories
for split, images in zip(["train", "val", "test"], [train_images, val_images, test_images]):
    split_dir = os.path.join(data_dir, split)
    os.makedirs(split_dir, exist_ok=True)
    os.makedirs(os.path.join(split_dir, "images"), exist_ok=True)
    os.makedirs(os.path.join(split_dir, "labels"), exist_ok=True)
    for image_file in images:
        shutil.copy(os.path.join(images_dir, image_file), os.path.join(split_dir, "images"))
        label_file = image_file.replace(".jpg", ".txt")
        shutil.copy(os.path.join(labels_dir, label_file), os.path.join(split_dir, "labels"))

Error: Destination path '/content/drive/MyDrive/traffic/ts/ts/images/00000.jpg' already exists

In [ ]:
import yaml

data_yaml = {
    'train': '/content/drive/MyDrive/traffic/ts/ts/train',
    'val': '/content/drive/MyDrive/traffic/ts/ts/val',
    'test': '/content/drive/MyDrive/traffic/ts/ts/test',
    'nc': 4,  # Update with the actual number of classes
    'names': ['prohibitory', 'danger', 'mandatory', 'other']  # Update with your actual class names
}

with open('/content/drive/MyDrive/traffic/ts/ts/data.yaml', 'w') as f:
    yaml.dump(data_yaml, f, indent=2)

In [ ]:
!python /content/yolov5/train.py --img 640 --batch 32 --epochs 30 --data /content/drive/MyDrive/traffic/ts/ts/data.yaml --weights yolov5s.pt --cache


wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2024-11-29 17:35:58.999217: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-29 17:35:59.037674: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-29 17:35:59.049119: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visua

In [ ]:
!python /content/yolov5/detect.py --weights ../../../yolov5/runs/train/exp/weights/best.pt --img 640 --conf 0.2 --source /content/drive/MyDrive/traffic/ts/ts/val/images/00005.jpg

detect: weights=['../../../yolov5/runs/train/exp/weights/best.pt'], source=/content/drive/MyDrive/traffic/ts/ts/val/images/00005.jpg, data=../../../yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.2, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=../../../yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-388-g882c35fc Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
WARNING ⚠️ NMS time limit 0.550s exceeded
image 1/1 /content/drive/MyDrive/traffic/ts/ts/val/images/00005.jpg: 384x640 1 prohibitory, 55.7ms
Speed: 0.8ms pre-process, 55.7ms inference, 1275.2ms NMS per image at shape (1, 3, 640, 

My data


In [ ]:
%cd /content/drive/MyDrive/ownData

/content/drive/MyDrive/ownData


In [ ]:
!unzip /content/drive/MyDrive/ownData/hand1-2.v1i.yolov5pytorch.zip

Archive:  /content/drive/MyDrive/ownData/hand1-2.v1i.yolov5pytorch.zip
  inflating: README.dataset.txt      
  inflating: README.roboflow.txt     
  inflating: data.yaml               
   creating: test/
   creating: test/images/
 extracting: test/images/20241130_215055_jpg.rf.ee88565e0ef3a5b319c56055b106e0fa.jpg  
 extracting: test/images/20241130_215105_jpg.rf.3b9c3281d05716292dbd8723f0842d66.jpg  
 extracting: test/images/20241130_215107_jpg.rf.64208383f725ddb3e089aea158195f63.jpg  
 extracting: test/images/20241130_215113_jpg.rf.46c5a7e9a5fc4346e94bd12b40c94cd1.jpg  
 extracting: test/images/20241130_215214_jpg.rf.ca85295bb3509cac6ee73e79c385aadd.jpg  
   creating: test/labels/
  inflating: test/labels/20241130_215055_jpg.rf.ee88565e0ef3a5b319c56055b106e0fa.txt  
  inflating: test/labels/20241130_215105_jpg.rf.3b9c3281d05716292dbd8723f0842d66.txt  
  inflating: test/labels/20241130_215107_jpg.rf.64208383f725ddb3e089aea158195f63.txt  
  inflating: test/labels/20241130_215113_jpg.rf.

In [ ]:
!python /content/yolov5/train.py --img 416 --batch 32 --epochs 50 --data /content/drive/MyDrive/ownData/data.yaml --weights yolov5l.pt --cache


wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2024-12-02 14:47:57.772343: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-02 14:47:57.791573: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-02 14:47:57.797436: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visua